# Setup

In [ ]:
!pip install graphdatascience

     |████████████████████████████████| 89 kB 3.9 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for neo4j: filename=neo4j-4.4.2-py3-none-any.whl size=115365 sha256=3224f38a507435979e0b5832e03193e5b4da4eb113d5ef414c688c09385ca49c
  Stored in directory: /root/.cache/pip/wheels/10/d6/28/95029d7f69690dbc3b93e4933197357987de34fbd44b50a0e4
Successfully built neo4j


In [ ]:
# Client import
from graphdatascience import GraphDataScience

# Connection details
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
AURA_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = ""

# Configures the client with AuraDS-recommended settings
gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD),
    aura_ds=True
)

In [ ]:
# Imported to prettify some results
import json

# Imported for the JSON helper function
from neo4j.time import DateTime

# Helper function for serialising Neo4j DateTime in JSON dumps
def default(o):
    if isinstance(o, (DateTime)):
        return o.isoformat()

# Create constraints

In [ ]:
# Make movie titles unique
gds.run_cypher("""
    CREATE CONSTRAINT FOR (movie:Movie) REQUIRE movie.title IS UNIQUE
""")

# Make person names unique
gds.run_cypher("""
    CREATE CONSTRAINT FOR (person:Person) REQUIRE person.name IS UNIQUE
""")

[]

# Load CSV files

In [ ]:
# Use "MERGE" on indexed label to take advantage of the constraint 
# while creating nodes.
# Use "ON CREATE SET" to set properties on created nodes.
# Use "RETURN count(*)" to show the number of processed rows.
gds.run_cypher("""
    LOAD CSV
      WITH HEADERS
      FROM 'https://data.neo4j.com/intro/movies/movies.csv' AS row
    MERGE (m:Movie {title: row.title})
      ON CREATE SET m.released = toInteger(row.released), m.tagline = row.tagline
    RETURN count(*)
""")

[{'count(*)': 38}]

In [ ]:
# Use "MERGE" on indexed label to take advantage of the constraint 
# while creating nodes.
# Use "ON CREATE SET" to set properties on created nodes.
# Use "RETURN count(*)" to show the number of processed rows.
gds.run_cypher("""
    LOAD CSV
      WITH HEADERS
      FROM 'https://data.neo4j.com/intro/movies/people.csv' AS row
    MERGE (p:Person {name: row.name}) 
      ON CREATE SET p.born = toInteger(row.born)
    RETURN count(*)
""")

[{'count(*)': 102}]

In [ ]:
# Use "USING PERIODIC COMMIT" to control the usage of memory.
# Use FIELDTERMINATOR to explicitly set the terminator character.
# Use "MATCH" and "MERGE" to create relationships between matched nodes.
# Use "ON CREATE SET" to set properties on created relationships.
# Use "RETURN count(*)" to show the number of processed rows.
gds.run_cypher("""
    USING PERIODIC COMMIT 50
    LOAD CSV
      WITH HEADERS
      FROM 'https://data.neo4j.com/intro/movies/actors.csv' AS row
      FIELDTERMINATOR ','
    MATCH (p:Person {name: row.person})
    MATCH (m:Movie {title: row.movie})
    MERGE (p)-[actedIn:ACTED_IN]->(m)
      ON CREATE SET actedIn.roles = split(row.roles, ';')
    RETURN count(*)
""")

[{'count(*)': 172}]

In [ ]:
gds.run_cypher("""
    MATCH (person:Person {name: "Keanu Reeves"})-[:ACTED_IN]->(movie)
    RETURN movie.released, COLLECT(movie.title) AS movies
    ORDER BY movie.released
""")

[{'movie.released': 1975, 'movies': ["Something's Gotta Give"]},
 {'movie.released': 1995, 'movies': ['Johnny Mnemonic']},
 {'movie.released': 1997, 'movies': ["The Devil's Advocate"]},
 {'movie.released': 1999, 'movies': ['The Matrix']},
 {'movie.released': 2000, 'movies': ['The Replacements']},
 {'movie.released': 2003,
  'movies': ['The Matrix Reloaded', 'The Matrix Revolutions']}]

# Cleanup

In [ ]:
# Delete data
gds.run_cypher("""
    MATCH (n)
    DETACH DELETE n
""")

[]